In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
# !pip install html5lib 
# df = pd.read_html('http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2021.csv')

In [6]:
# Extract Data
col_names = ['tid', 'price', 'dateOfTransfer', 'postcode', 'propertyType', 'isNewProperty', 'tenure', 'paon', 'saon', 'street', 'locality', 'city', 'district',
             'country', 'ttype', 'recordStatus'] # 'yearOfTransfer', 'monthOfTransfer', 'pc_left', 'pc_right'
df = pd.read_csv('E:\learn\interview\data\pp-2021.csv', header = None,  names = col_names  )
df.head()

,tid,price,dateOfTransfer,postcode,propertyType,isNewProperty,tenure,paon,saon,street,locality,city,district,country,ttype,recordStatus
0,{E53EDD2E-C6F0-83EC-E053-6B04A8C03A59},70000,2021-06-11 00:00,HX2 8SN,T,N,F,16,NaN,CLOUGH LANE,MIXENDEN,HALIFAX,CALDERDALE,WEST YORKSHIRE,B,A
1,{E53EDD2E-C6F1-83EC-E053-6B04A8C03A59},150000,2021-11-08 00:00,LS27 9AL,O,N,F,RUSSELL HOUSE,3,HIGH STREET,MORLEY,LEEDS,LEEDS,WEST YORKSHIRE,B,A
2,{E53EDD2E-C6F3-83EC-E053-6B04A8C03A59},430000,2021-10-29 00:00,LS6 1BU,D,N,F,10,NaN,MANOR TERRACE,HEADINGLEY,LEEDS,LEEDS,WEST YORKSHIRE,B,A
3,{E53EDD2E-C6F4-83EC-E053-6B04A8C03A59},135000,2021-10-26 00:00,LS10 1LP,F,N,L,MCCLINTOCK HOUSE,APARTMENT 32,THE BOULEVARD,NaN,LEEDS,LEEDS,WEST YORKSHIRE,B,A
4,{E53EDD2E-C704-83EC-E053-6B04A8C03A59},131000,2021-12-21 00:00,HD2 2SN,O,N,F,244,NaN,HALIFAX OLD ROAD,BIRKBY,HUDDERSFIELD,KIRKLEES,WEST YORKSHIRE,B,A


In [108]:
df.shape


(1278278, 16)

In [113]:
temp = df[['postcode', 'paon', 'saon', 'street', 'locality', 'city']].drop_duplicates()
temp.shape

(1260478, 6)

In [111]:
temp.loc[~temp['postcode'].isnull()]

,postcode,paon,saon,street,locality,city
0,HX2 8SN,16,NaN,CLOUGH LANE,MIXENDEN,HALIFAX
1,LS27 9AL,RUSSELL HOUSE,3,HIGH STREET,MORLEY,LEEDS
2,LS6 1BU,10,NaN,MANOR TERRACE,HEADINGLEY,LEEDS
3,LS10 1LP,MCCLINTOCK HOUSE,APARTMENT 32,THE BOULEVARD,NaN,LEEDS
4,HD2 2SN,244,NaN,HALIFAX OLD ROAD,BIRKBY,HUDDERSFIELD
...,...,...,...,...,...,...
1278273,NN2 7RA,40,NaN,WEST RIDGE,NaN,NORTHAMPTON
1278274,NN11 9AU,10,NaN,COLERIDGE WALK,NaN,DAVENTRY
1278275,NN1 3JE,39,NaN,CLARE STREET,NaN,NORTHAMPTON
1278276,NN18 8QS,5,NaN,GOODWOOD CLOSE,OAKLEY,CORBY


In [118]:
year = 2021
url = f'http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-{year}.csv'
print(url)
filename = url.split('/')[-1]
print(filename)
output_path = f'data\{filename}'
output_path

http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2021.csv
pp-2021.csv


'data\\pp-2021.csv'

In [ ]:
# Cleaning Data

# 1. Remove Rows with null post codes
df_q = df.loc[~ df['postcode'].isnull(), ['dateOfTransfer','postcode', 'price']]
# Handling Date
df['dateOfTransfer'] = pd.to_datetime(df['dateOfTransfer'])
# Derive post code area
df_q['postcode_area'] = df_q['postcode'].apply(lambda x: x.split(' ')[0])
# Extract Year
df_q['year'] = df_q['dateOfTransfer'].dt.year

In [119]:
# Questions
# 1. What are the top 10 postcode areas (for example CB22, DL4, …) with the lowest yearly median house price paid in the year 2021?
postaArea_medianPrice_df = df_q.groupby('postcode_area')['price'].median().sort_values()
postaArea_medianPrice_df.iloc[:10].index

Index(['DL4', 'TS1', 'BD1', 'DL17', 'SR1', 'SR8', 'DN31', 'BB11', 'TS3',
       'TS29'],
      dtype='object', name='postcode_area')

In [120]:
postaArea_medianPrice_df.iloc[:10].index.to_list()

['DL4', 'TS1', 'BD1', 'DL17', 'SR1', 'SR8', 'DN31', 'BB11', 'TS3', 'TS29']

In [8]:
# 2. What are the top 5 postcode areas prefixed with CB and with the smallest yearly mean house price in the year 2021?
pattern = 'CB'
pattern_match = df_q['postcode_area'].str.startswith(pattern)

CBpostaArea_meanPrice_df = df_q.loc[pattern_match]
CBpostaArea_meanPrice_df.groupby('postcode_area')['price'].mean().sort_values().iloc[:5].index

Index(['CB9', 'CB6', 'CB7', 'CB25', 'CB24'], dtype='object', name='postcode_area')

In [98]:
year = 2021
url = f'http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-{year}.csv'
output_path = f'data\pp-{year}.csv'

# url = "https://example.com/data.csv"

# with requests.get(url, stream=True) as r:
#     r.raise_for_status()  # Raise error if download fails
#     with open(output_path, "wb") as f:
#         for chunk in r.iter_content(chunk_size=8192):  # 1MB chunks
#             f.write(chunk)

# print(f"Downloaded to {output_path}")

response = requests.get(url)
with open(output_path, "wb") as f:
    f.write(response.content)

# import pandas as pd
# df = pd.read_csv("local.csv")

In [ ]:
# df_iterator = pd.read_csv(url, chunksize=100000)
# for chunk in df_iterator:

In [92]:
df_temp = pd.read_csv(f'E:\learn\interview\data\pp-{2021}.csv', header = None,  names = col_names)

In [123]:
# 3. What are the top overall 25 postcode areas with the lowest cumulative yearly median difference sum house price since registered by 1995 up to 2021?
start_year = 1995
end_year = 2021
years = [y for y in range(start_year,end_year+1)]

df = pd.DataFrame(columns= col_names)
for year in years:
    print(year)
    df_temp = pd.read_csv(f'E:\learn\interview\data_1\pp-{year}.csv', header = None,  names = col_names)
    df = pd.concat([df, df_temp], ignore_index = True)
    # print(df.shape)



1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [ ]:
df = df.loc[~ df['postcode'].isnull()]
df['postcode_area'] = df['postcode'].astype(str).apply(lambda x: x.split(' ')[0])
df['dateOfTransfer'] = pd.to_datetime(df['dateOfTransfer'])
df['year'] = df['dateOfTransfer'].dt.year

In [57]:
# df[['postcode_area', 'year']].drop_duplicates().groupby('postcode_area').count() == 27

,year
postcode_area,
AL1,True
AL10,True
AL2,True
AL3,True
AL4,True
...,...
YO62,True
YO7,True
YO8,True


In [ ]:
df_q3 = df.groupby(['postcode_area', 'year'])['price'].median()
df_q3= df_q3.reset_index(drop = False)
df_q3['price_diff'] = df_q3.groupby(['postcode_area'])['price'].diff()
df_q3['price_diff']= df_q3['price_diff'].fillna(0)
df_q3.groupby('postcode_area')['price_diff'].sum().sort_values().iloc[:25]

KeyError: 'postcode_area'

In [ ]:
# df_q3 = df_q3.sort_values(by=['postcode_area', 'year'])

In [70]:
df_q3[df_q3['postcode_area'] == 'UB11']

,postcode_area,year,price,price_diff
57200,UB11,2013,20900000.0,0.0
57201,UB11,2014,15750000.0,-5150000.0
57202,UB11,2015,7220631.5,-8529368.5
57203,UB11,2016,21600000.0,14379368.5
57204,UB11,2017,21400000.0,-200000.0
57205,UB11,2018,17550000.0,-3850000.0
57206,UB11,2020,20343750.0,2793750.0
57207,UB11,2021,4750000.0,-15593750.0


In [ ]:
# df_temp_3 = df_q3.copy()
# df_temp_3['year_next'] = df_temp_3['year'] + 1

In [66]:
# df_temp_3

,postcode_area,year,price,year_next
0,AL1,1995,79450.0,1996
1,AL1,1996,83500.0,1997
2,AL1,1997,99700.0,1998
3,AL1,1998,109850.0,1999
4,AL1,1999,129950.0,2000
...,...,...,...,...
62011,YO8,2021,210000.0,2022
62012,YO90,2015,28100000.0,2016
62013,YO90,2016,12750000.0,2017
62014,YO90,2017,15500000.0,2018


In [75]:
# df_q3_1 = pd.merge(df_q3, df_temp_3, left_on= ['year', 'postcode_area'], right_on= ['year_next', 'postcode_area'], how = 'left', suffixes= ['_original', '__prev_year'])
# df_q3_1['median_diff'] = df_q3_1['price_original'] - df_q3_1['price__prev_year']

0               NaN
1            4050.0
2           16200.0
3           10150.0
4           20100.0
            ...    
62011       15000.0
62012           NaN
62013   -15350000.0
62014     2750000.0
62015           NaN
Length: 62016, dtype: object

In [45]:
# df_q3.sort_values(by = ['postcode_area', 'year'])

,postcode_area,year,price
0,AL1,1995,79450.0
1,AL1,1996,83500.0
2,AL1,1997,99700.0
3,AL1,1998,109850.0
4,AL1,1999,129950.0
...,...,...,...
59710,YO8,2020,195000.0
59711,YO90,2015,28100000.0
59712,YO90,2016,12750000.0
59713,YO90,2017,15500000.0


In [ ]:
'''
4. If we filter by postcode area prefix “CB”, take the top 5 smallest yearly median price postcode areas and sort them by the cumulative yearly 
mean difference sum price in ascending order, what is the result of it?
'''

In [75]:
pattern = 'CB'
pattern_match = df['postcode_area'].str.startswith(pattern)
CBpostaArea_medianPrice_df = df.loc[pattern_match]

CBpostaArea_medianPrice_df.groupby(['postcode_area', 'year'])['price'].median()

postcode_area  year
CB1            1995     69995.0
               1996     75000.0
               1997     84500.0
               1998     92000.0
               1999    104475.0
                         ...   
CB9            2017    237750.0
               2018    245000.0
               2019    245000.0
               2020    245000.0
               2021    268000.0
Name: price, Length: 434, dtype: object

In [83]:
df_q4 = CBpostaArea_medianPrice_df.groupby(['postcode_area', 'year'])['price'].mean()
df_q4= df_q4.reset_index(drop = False)
df_q4

,postcode_area,year,price
0,CB1,1995,77718.197368
1,CB1,1996,82896.135447
2,CB1,1997,92611.487611
3,CB1,1998,101570.008239
4,CB1,1999,117586.158379
...,...,...,...
429,CB9,2017,303157.572695
430,CB9,2018,290779.872211
431,CB9,2019,295042.286282
432,CB9,2020,278515.313609


In [84]:
df_q4['price_diff'] = df_q4.groupby(['postcode_area'])['price'].diff()
df_q4['price_diff']= df_q4['price_diff'].fillna(0)
df_q4

,postcode_area,year,price,price_diff
0,CB1,1995,77718.197368,0.000000
1,CB1,1996,82896.135447,5177.938078
2,CB1,1997,92611.487611,9715.352164
3,CB1,1998,101570.008239,8958.520628
4,CB1,1999,117586.158379,16016.150140
...,...,...,...,...
429,CB9,2017,303157.572695,71486.663604
430,CB9,2018,290779.872211,-12377.700484
431,CB9,2019,295042.286282,4262.414071
432,CB9,2020,278515.313609,-16526.972673


In [89]:

df_q4.groupby('postcode_area')['price_diff'].sum().sort_values().iloc[:25].apply(lambda x: int(x))

postcode_area
CB9      248126
CB6      269518
CB7      281720
CB25     339044
CB24     352465
CB8      383917
CB21     418681
CB23     424142
CB10     460884
CB1      491835
CB11     508645
CB22     516316
CB5      529486
CB2      547519
CB4      767534
CB3      785246
CB20    6241500
Name: price_diff, dtype: int64

In [4]:
col_names = ['tid', 'price', 'dateOfTransfer', 'postcode', 'propertyType', 'isNewProperty', 'tenure', 'paon', 'saon', 'street', 'locality', 'city', 'district',
             'country', 'ttype', 'recordStatus']
year = 2021
import pandas as pd
from pathlib import Path
file_path = Path("..") / "data" / f"pp-{year}.csv"
df_temp = pd.read_csv(file_path, header=None, names=col_names)  # Unix or portable


In [5]:
df_temp

,tid,price,dateOfTransfer,postcode,propertyType,isNewProperty,tenure,paon,saon,street,locality,city,district,country,ttype,recordStatus
0,{E53EDD2E-C6F0-83EC-E053-6B04A8C03A59},70000,2021-06-11 00:00,HX2 8SN,T,N,F,16,NaN,CLOUGH LANE,MIXENDEN,HALIFAX,CALDERDALE,WEST YORKSHIRE,B,A
1,{E53EDD2E-C6F1-83EC-E053-6B04A8C03A59},150000,2021-11-08 00:00,LS27 9AL,O,N,F,RUSSELL HOUSE,3,HIGH STREET,MORLEY,LEEDS,LEEDS,WEST YORKSHIRE,B,A
2,{E53EDD2E-C6F3-83EC-E053-6B04A8C03A59},430000,2021-10-29 00:00,LS6 1BU,D,N,F,10,NaN,MANOR TERRACE,HEADINGLEY,LEEDS,LEEDS,WEST YORKSHIRE,B,A
3,{E53EDD2E-C6F4-83EC-E053-6B04A8C03A59},135000,2021-10-26 00:00,LS10 1LP,F,N,L,MCCLINTOCK HOUSE,APARTMENT 32,THE BOULEVARD,NaN,LEEDS,LEEDS,WEST YORKSHIRE,B,A
4,{E53EDD2E-C704-83EC-E053-6B04A8C03A59},131000,2021-12-21 00:00,HD2 2SN,O,N,F,244,NaN,HALIFAX OLD ROAD,BIRKBY,HUDDERSFIELD,KIRKLEES,WEST YORKSHIRE,B,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278273,{D707E536-89CA-0AD9-E053-6B04A8C067CC},158000,2021-12-17 00:00,NN2 7RA,T,N,F,40,NaN,WEST RIDGE,NaN,NORTHAMPTON,WEST NORTHAMPTONSHIRE,WEST NORTHAMPTONSHIRE,B,A
1278274,{D707E536-89CB-0AD9-E053-6B04A8C067CC},105000,2021-05-26 00:00,NN11 9AU,F,N,L,10,NaN,COLERIDGE WALK,NaN,DAVENTRY,WEST NORTHAMPTONSHIRE,WEST NORTHAMPTONSHIRE,B,A
1278275,{D707E536-89CC-0AD9-E053-6B04A8C067CC},239000,2021-10-05 00:00,NN1 3JE,T,N,F,39,NaN,CLARE STREET,NaN,NORTHAMPTON,WEST NORTHAMPTONSHIRE,WEST NORTHAMPTONSHIRE,B,A
1278276,{D707E536-89CD-0AD9-E053-6B04A8C067CC},190000,2021-09-08 00:00,NN18 8QS,T,N,F,5,NaN,GOODWOOD CLOSE,OAKLEY,CORBY,NORTH NORTHAMPTONSHIRE,NORTH NORTHAMPTONSHIRE,B,A


In [16]:

df = df.loc[~ df['postcode'].isnull()]
df['postcode_area'] = df['postcode'].astype(str).apply(lambda x: x.split(' ')[0])
counts_series = df['postcode_area'].value_counts()
postcode_areas_cutoff = counts_series[counts_series >= 10].index.to_list()
postcode_areas_cutoff

['E14',
 'CR0',
 'BN3',
 'SW11',
 'ST5',
 'E17',
 'BN1',
 'BS16',
 'BN2',
 'PR4',
 'SW18',
 'LE12',
 'LU7',
 'SL6',
 'LE2',
 'SW19',
 'PR2',
 'NG5',
 'LE3',
 'PE7',
 'LE67',
 'B90',
 'GL2',
 'NG9',
 'CF14',
 'TQ12',
 'BA2',
 'PR7',
 'PR3',
 'CM1',
 'CV6',
 'NG17',
 'SW16',
 'RH10',
 'BH23',
 'NN3',
 'LN6',
 'WA5',
 'RH12',
 'FY5',
 'LU2',
 'RH1',
 'SW17',
 'FY8',
 'M33',
 'CM2',
 'ST4',
 'ST7',
 'LE9',
 'SS9',
 'DN4',
 'PE28',
 'SK8',
 'PR8',
 'SW6',
 'BL1',
 'CF5',
 'NG16',
 'BB5',
 'MK45',
 'CV37',
 'GL52',
 'CO10',
 'PE19',
 'NG24',
 'LE10',
 'PE2',
 'CM3',
 'CO15',
 'N1',
 'NW9',
 'OX16',
 'TA6',
 'CO4',
 'CV10',
 'DA1',
 'NG2',
 'RG12',
 'SS7',
 'SO50',
 'WA4',
 'WN7',
 'DN14',
 'S6',
 'CV3',
 'HU17',
 'SE1',
 'PE11',
 'NR31',
 'YO8',
 'ST6',
 'DE13',
 'LE4',
 'SK9',
 'MK42',
 'IP14',
 'M5',
 'NP19',
 'SK6',
 'DE55',
 'CW2',
 'LE16',
 'PR1',
 'WF10',
 'LE7',
 'SN25',
 'GU14',
 'ME10',
 'NN14',
 'CM7',
 'NW10',
 'N4',
 'W5',
 'DE22',
 'NN4',
 'DE11',
 'BN11',
 'B77',
 'ST3',
 'LS25